In [5]:
import datetime
import json
import os
import pickle
from dateutil import parser

import gradio as gr 
from openai import OpenAI
import keyring

OPENAI_API_KEY = keyring.get_password('openai', 'key_for_mac')
MAPPINGS = {
    '서귀포': './res/yanolja.json',
    '판교': './res/ninetree_pangyo.json',
    '용산': './res/ninetree_yongsan.json'
}

with open('./res/prompt_1shot.pickle', 'rb') as f:
    PROMPT = pickle.load(f)
    

In [6]:
def preprocess_reviews(path='./res/yanolja.json'):
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)
        
    reviews_good, reviews_bad = [], []
    
    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)
    
    filtered_cnt = 0
    for r in review_list:
        review_date_str = r['date']
        try:
            review_date = parser.parse(review_date_str)
            
        except (ValueError, TypeError):
            review_date = current_date
            
        if review_date < date_boundary:
            continue
        if len(r['review']) < 30:
            filtered_cnt += 1
            continue
        
        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
            
    reviews_good = reviews_good[:min(len(reviews_good), 50)]
    reviews_bad = reviews_bad[:min(len(reviews_bad), 50)]
    
    reviews_good_text = '\n'.join(reviews_good)
    review_bad_text = '\n'.join(reviews_bad)
    
    return reviews_good_text, review_bad_text


In [7]:
def summarize(reviews):
    prompt = PROMPT + '\n\n' + reviews
    
    client = OpenAI(api_key=keyring.get_password('openai', 'key_for_mac'))
    completion = client.chat.completions.create(
        model='gpt-3.5-turbo-0125',
        messages=[{'role': 'user', 'content': prompt}],
        temperature=0.0
    )
    
    return completion

In [8]:
def fn(accom_name):
    path = MAPPINGS[accom_name]
    reviews_good, reviews_bad = preprocess_reviews(path)
    
    summary_good = summarize(reviews_good).choices[0].message.content
    summary_bad = summarize(reviews_bad).choices[0].message.content
    
    return summary_good, summary_bad

In [9]:
def run_demo():
    demo = gr.Interface(
        fn=fn,
        inputs=[gr.Radio(['서귀포', '판교', '용산'], label='숙소')],
        outputs=[gr.Textbox(label='높은 평점 요약'), gr.Textbox(label='낮은 평점 요약')]
    )
    demo.launch(share=True)

run_demo()

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://7501bb90b54ff62773.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
